# Experiments driver for HuGaDB

This notebook collects the experiments performed for TIME 2025.

If you are running the code of each cell in the REPL, please make sure your working
directory is the same as this file by executing `; pwd`, or adjust the relative paths
accordingly to your system.

More about the dataset on https://github.com/romanchereshnev/HuGaDB.

In [1]:
import Pkg
using Serialization: serialize, deserialize

# activate the general environment of ModalAssociationRules.jl package
Pkg.activate(joinpath("..", "..", ".."))

# a collection of utilities
include(joinpath("..", "experiments-driver.jl"));

  Activating project at `~/.julia/dev/ModalAssociationRules`


In [ ]:
# algorithm to be used for mining
miningalgo = apriori

# distance function to compare a time series and a motif of the same length; 
# we suggest to use Z-Normalized Euclidean distance if you are not interested in the scale
# but just in identifying common shapes between signals.
expdistance = (x, y) -> zeuclidean(x, y) |> first;

# variables considered for these experiments;
# see the anatomic schema at https://github.com/romanchereshnev/HuGaDB
expvars = [ # accelerometer data
    1,3, # right foot x (+behind/-front) and z (+down/-up)  
    13,15, # right thigh x (+up/-down) and z (+behind/-front)
    19,21, # left foot x (+behind/-front) and z (+down/-up)
    31,33  # left thigh x (+up/-down) and z (+behind/-front)
]

# files selected to build the experiments DataFrame
expfiles = ["HuGaDB_v2_various_02_00.txt", "HuGaDB_v2_various_02_01.txt", 
    "HuGaDB_v2_various_02_02.txt", "HuGaDB_v2_various_02_03.txt", "HuGaDB_v2_various_02_04.txt", 
    "HuGaDB_v2_various_02_05.txt", "HuGaDB_v2_various_02_06.txt", "HuGaDB_v2_various_06_00.txt", 
    "HuGaDB_v2_various_06_01.txt", "HuGaDB_v2_various_06_02.txt", "HuGaDB_v2_various_06_03.txt", 
    "HuGaDB_v2_various_06_04.txt", "HuGaDB_v2_various_06_05.txt", "HuGaDB_v2_various_06_06.txt", 
    "HuGaDB_v2_various_06_07.txt", "HuGaDB_v2_various_06_08.txt", "HuGaDB_v2_various_06_09.txt", 
    "HuGaDB_v2_various_06_10.txt", "HuGaDB_v2_various_06_11.txt", "HuGaDB_v2_various_06_12.txt", 
    "HuGaDB_v2_various_06_13.txt", "HuGaDB_v2_various_06_14.txt", "HuGaDB_v2_various_06_15.txt", 
    "HuGaDB_v2_various_06_16.txt",  "HuGaDB_v2_various_06_17.txt", "HuGaDB_v2_various_06_18.txt", 
    "HuGaDB_v2_various_06_19.txt",  "HuGaDB_v2_various_06_20.txt", "HuGaDB_v2_various_06_21.txt", 
    "HuGaDB_v2_various_06_22.txt", "HuGaDB_v2_various_06_23.txt", "HuGaDB_v2_various_06_24.txt", 
    "HuGaDB_v2_various_06_25.txt","HuGaDB_v2_various_06_26.txt", "HuGaDB_v2_various_06_27.txt"
];

35-element Vector{String}:
 "HuGaDB_v2_various_02_00.txt"
 "HuGaDB_v2_various_02_01.txt"
 "HuGaDB_v2_various_02_02.txt"
 "HuGaDB_v2_various_02_03.txt"
 "HuGaDB_v2_various_02_04.txt"
 "HuGaDB_v2_various_02_05.txt"
 "HuGaDB_v2_various_02_06.txt"
 "HuGaDB_v2_various_06_00.txt"
 "HuGaDB_v2_various_06_01.txt"
 "HuGaDB_v2_various_06_02.txt"
 ⋮
 "HuGaDB_v2_various_06_19.txt"
 "HuGaDB_v2_various_06_20.txt"
 "HuGaDB_v2_various_06_21.txt"
 "HuGaDB_v2_various_06_22.txt"
 "HuGaDB_v2_various_06_23.txt"
 "HuGaDB_v2_various_06_24.txt"
 "HuGaDB_v2_various_06_25.txt"
 "HuGaDB_v2_various_06_26.txt"
 "HuGaDB_v2_various_06_27.txt"

In [3]:
# load all the data
X, (activity_strings, activity_ids), variablenames = load_hugadb(expfiles)

(35×39 DataFrame
 Row │ acc_rf_x                           acc_rf_y                           a ⋯
     │ Array…                             Array…                             A ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ [-7504.0, -7476.0, -7532.0, -750…  [-5096.0, -5076.0, -5040.0, -510…  [ ⋯
   2 │ [-7312.0, -7320.0, -7300.0, -739…  [-4920.0, -4776.0, -4896.0, -489…  [
   3 │ [-7312.0, -7252.0, -7256.0, -730…  [-4508.0, -4528.0, -4500.0, -448…  [
   4 │ [-7348.0, -7432.0, -7388.0, -740…  [-4548.0, -4588.0, -4572.0, -454…  [
   5 │ [-7156.0, -7192.0, -7144.0, -714…  [-4520.0, -4524.0, -4444.0, -444…  [ ⋯
   6 │ [-6984.0, -32768.0, -29976.0, -3…  [-8204.0, -13772.0, -21536.0, -1…  [
   7 │ [-7928.0, -7636.0, -7788.0, -770…  [-5464.0, -5500.0, -5512.0, -583…  [
   8 │ [-8292.0, -8208.0, -8216.0, -833…  [-4884.0, -5016.0, -5072.0, -510…  [
  ⋮  │                 ⋮                                  ⋮                    ⋱
  29 │ [-7108.0, -7236.

## Walking

In [4]:
X_walking = filter_hugadb(X, 1) # 1 is the activity id for walking
X_walking = X_walking[:,expvars]

# uncomment if you want to make experiments more lightweights
for i in 1:ninstances(X_walking)
    for j in 1:nvariables(X_walking)
        X_walking[i,j] = X_walking[i,j][100:199]
    end
end

In [5]:
# the lenght of the first two snippets extracted is 25
_n1 = 2     # keep 2 if you want more lightweight experiments,
            # howver it is interesting to enlarge this, making the extraction more granular
_m1 = 25

# the lenghth of the other two snippets extracted is 50
_n2 = 2
_m2 = 50

# needed later to build a Miner which only mines interval of these lengths
_mcoherence = x -> length(x) in [_m1, _m2]

#41 (generic function with 1 method)

In [ ]:
# run this cell if you need to create your literals, that is,
# to extract snippets from time series and give them a name.

ids_walking, motifs_walking, featurenames_walking = label_motifs(
    X_walking, collect(1:length(expvars)), variablenames[expvars], "HuGaDB-Walking";
    m1=_m1, n1=_n1,   # we want _n1 snippets of length _m1
    m2=_m2, n2=_n2    # and _n2 snippets of length _m2
)

In [7]:
# run this cell if you already created your literals
ids_walking, motifs_walking, featurenames_walking = load_motifs("HuGaDB-Walking");

In [ ]:
# run the experiment (hyperparameters are `initialize_experiment` kwargs) & save the results
logiset, miner = initialize_experiment(
    ids_walking, motifs_walking, featurenames_walking, X_walking;
    _worldfilter=SoleLogics.FunctionalWorldFilter(_mcoherence, Interval{Int}),
    _itemsetmeasures = [(gsupport, 0.1, 0.1)],
)
experiment!(miner, "walking")

Mining...
Mining duration: 2912.19
Generating rules...
Generation duration: 634.22
Writing to: /home/mauro/.julia/dev/ModalAssociationRules/test/experiments/results/i_have_command


# Running

In [ ]:
X_running = filter_hugadb(X, 2) # 2 is the activity id for running
X_running = X_running[:,expvars]

# uncomment if you want to make experiments more lightweights
for i in 1:ninstances(X_running)
    for j in 1:nvariables(X_running)
        X_running[i,j] = X_running[i,j][100:199]
    end
end

In [ ]:
ids_running, motifs_running, featurenames_running = label_motifs(
    X_running, collect(1:length(expvars)), variablenames[expvars], "HuGaDB-Running";
    m1=_m1, n1=_n1,   # we want _n1 snippets of length _m1
    m2=_m2, n2=_n2    # and _n2 snippets of length _m2
)

In [9]:
# run this cell if you already created your literals
ids_running, motifs_running, featurenames_running = load_motifs("HuGaDB-Running");

In [ ]:
# run the experiment (hyperparameters are `initialize_experiment` kwargs) & save the results
logiset, miner = initialize_experiment(
    ids_running, motifs_running, featurenames_running, X_running;
    _worldfilter=SoleLogics.FunctionalWorldFilter(_mcoherence, Interval{Int}),
    _itemsetmeasures = [(gsupport, 0.1, 0.1)],
)
experiment!(miner, "running")

Mining...
Mining duration: 4826.88
Generating rules...
Generation duration: 1492.6
Writing to: /home/mauro/.julia/dev/ModalAssociationRules/test/experiments/results/not_clear


# Plots

In [10]:
using Plots
using PGFPlotsX
pgfplotsx()

Plots.PGFPlotsXBackend()

In [22]:
# example of two qualitatively similar motifs extracted from walking and running;
# they both encode the fact that the right thigh is going up and down, 
# but it is clear that in running class the acceleration are more sudden.

# they are both the 11th extracted motifs
mw11 = motifs_walking[11] |> first |> _normalize
mr11 = motifs_running[11] |> first |> _normalize
plot(
    mw11, 
    label=["Walking"], linecolor = :blue, linewidth = 1, aspect_ratio = :equal,
    titlefont=font(18, "Arial"), guidefontsize=14, size = (500, 300), ylims=(-2.5,2.5),
    legend = :topleft
);

plot!(
    mr11, 
    label=["Running"], linecolor = :red, linewidth = 1, aspect_ratio = :equal,
    titlefont=font(18, "Arial"), guidefontsize=14, size = (500, 300), ylims=(-2.5,2.5)
);

xlabel!("Time units")
ylabel!("Normalized acceleration")
title!("Example of motif for right thigh x")

savefig("results/plot_right_tigh_x.tex");